<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# default_exp data.cache

In [ ]:
# export

import os
from pathlib import Path

import sqlite3

# Try catch to resolve weirdness in GitHub actions runner.
try:
    CACHE_LOCATION = Path.home() / Path(".cache/uberduck/uberduck-ml-dev.db")
except:
    pass


def _path_to_speaker_name(path: str, speaker_idx_in_path=None):
    p = Path(path)
    if speaker_idx_in_path is not None:
        return p.parts[speaker_idx_in_path]
    assert "wavs" in p.parts, f"Can't autodetect speaker name from path: {p.parts}"
    wavs_idx = p.parts.index("wavs")
    return p.parts[wavs_idx - 1]


def ensure_speaker_table(database_path):
    db_path = Path(database_path)
    if not db_path.parent.exists():
        os.makedirs(db_path.parent)
    conn = sqlite3.connect(str(db_path))
    cursor = conn.cursor()
    sql = """CREATE TABLE IF NOT EXISTS filelists (uuid TEXT, 
            filelist_path TEXT, 
            speaker_name TEXT,
            speaker_id INT,
            dir_path TEXT,
            rel_path TEXT,
            dataset_name TEXT)
            """
    cursor.execute(sql)

In [ ]:
# skip
p = Path("/Users/zwf/data/voice/vapor-trail/wavs/vapor-trail-1.wav")
wavs_idx = p.parts.index("wavs")

In [ ]:
input_ = "/mnt/disks/uberduck-experiments-v0/data/uberduck-multispeaker/"

In [ ]:
# def insert_speaker(filelist, speaker, speaker_id, conn=None):
#     if conn is None:
#         conn = sqlite3.connect(str(CACHE_LOCATION))
#     conn.execute(
#         "INSERT OR REPLACE INTO speakers VALUES (?, ?, ?)",
#         (filelist, speaker, speaker_id),
#     )


#     with conn:
#         conn.execute(
#             '''CREATE TABLE IF NOT EXISTS filelists (UUID TEXT,
#             FILELIST_PATH TEXT,
#             SPEAKER_NAME TEXT,
#             SPEAKER_ID INT,
#             DIR_PATH TEXT,
#             REL_PATH TEXT,
#             DATASET_NAME TEXT)
#             '''
#         )
# #         conn.execute(
# #             "CREATE UNIQUE INDEX IF NOT EXISTS name_speaker_id_idx ON speakers(filepath, name)"
# #         )
# conn = sqlite3.connect(str(CACHE_LOCATION_EXP))
# cursor = conn.cursor()
# cursor.execute("DROP TABLE IF EXISTS FILELISTS")
# sql ='''CREATE TABLE FILELISTS(
#    UUID CHAR(50),
#    FILELIST_PATH CHAR(100),
#    SPEAKER_NAME CHAR(50),
#    SPEAKER_ID INT,
#    DIR_PATH CHAR(100),
#    REL_PATH CHAR(100),
#    DATASET_NAME CHAR(50)
# )'''
# cursor.execute(sql)

# def ensure_filelist_in_cache(filelist, speaker_idx_in_path=None):
#     with open(filelist) as f:
#         lines = f.readlines()
#     num_speakers = len(set([line.strip().split("|")[2] for line in lines]))
#     conn = sqlite3.connect(str(CACHE_LOCATION))
#     with conn:
#         count = conn.execute(
#             "SELECT COUNT(*) FROM speakers WHERE filepath = ?", (filelist,)
#         ).fetchone()[0]
#         speaker_name_to_id = {}
#         if count != num_speakers:
#             print(
#                 f"Filelist not found in cache. Cache count: {count}. Filelist count: {num_speakers}."
#             )
#             for line in lines:
#                 path, txn, sid = line.strip().split("|")
#                 speaker_name = _path_to_speaker_name(
#                     path, speaker_idx_in_path=speaker_idx_in_path
#                 )
#                 if speaker_name in speaker_name_to_id:
#                     continue
#                 else:
#                     speaker_name_to_id[speaker_name] = int(sid)
#             for speaker_name, speaker_id in speaker_name_to_id.items():
#                 insert_speaker(filelist, speaker_name, speaker_id, conn)